In [26]:
import pandas as pd
import numpy as np
import math
from functools import cmp_to_key

In [2]:
#mount your google drive.
#it will be visible in the file navigator on the left of this notebook
#there should be a folder in your drive with your data
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/Colab Notebooks

Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks


In [3]:
df = pd.read_csv('google-cluster-data-1.csv',sep=' ')
df

,Time,ParentID,TaskID,JobType,NrmlTaskCores,NrmlTaskMem,Unnamed: 6
0,90000,757745334,1488529826,0,0.000000,0.031130,NaN
1,90000,975992247,1488529821,0,0.000000,0.000000,NaN
2,90000,1468458091,1488529832,1,0.021875,0.002353,NaN
3,90000,1460281235,1488529840,0,0.000000,0.000000,NaN
4,90000,1164728954,1488529835,0,0.003125,0.001638,NaN
...,...,...,...,...,...,...,...
3535024,112500,1487094655,1487103476,0,0.000000,0.000879,NaN
3535025,112500,1461321601,1465612301,0,0.000000,0.000879,NaN
3535026,112500,1487094655,1487097223,0,0.000000,0.000879,NaN
3535027,112500,618817162,1485932004,1,0.000000,0.000879,NaN


## Append Execution Time and Turnaround Time information to dataframe
* Add total execution time list to dataframe under key "ExecutionTime". A task queue is created for each time quantum for which tasks are allocated. Tasks are allocated to a given task queue based on their arrival time and remaining execution time. Therefore, total execution time is used to determine which task queues each task should be placed in.
* Add execution time remaining list to datafram under key "ExecutionTimeRemaining". Execution time remaining is used by the SJF algorithm to determine which tasks should be allocated to the VMs first. 
* Add columns to store the turnaround times for each version of the greedy algorithm (power, cost, and just the turnaround time).

In [52]:
def powerVMAlloc(vms, task_idx, vm_idx, all_past_threshold):
  dy_pwr_threshold = 0.5
  
  cpu = df['NrmlTaskCores'][task_idx]

  # current vm calculation
  cpu_cap = 5.0
  ccr = float(vms[0][vm_idx])  # current core remaining 

  usage_rate_curr = 1.0 - (ccr/cpu_cap)

  if not all_past_threshold and df['NrmlTaskCores'][task_idx] <= vms[0][vm_idx] and df['NrmlTaskMem'][task_idx] <= vms[1][vm_idx]:
    if (usage_rate_curr < dy_pwr_threshold):
      vms[0][vm_idx] = vms[0][vm_idx] - df['NrmlTaskCores'][task_idx]
      vms[1][vm_idx] = vms[1][vm_idx] - df['NrmlTaskMem'][task_idx]
      return vms, True
    else:
      return vms, False
  elif all_past_threshold and df['NrmlTaskCores'][task_idx] <= vms[0][vm_idx] and df['NrmlTaskMem'][task_idx] <= vms[1][vm_idx]:
    vms[0][vm_idx] = vms[0][vm_idx] - df['NrmlTaskCores'][task_idx]
    vms[1][vm_idx] = vms[1][vm_idx] - df['NrmlTaskMem'][task_idx]
    return vms, True
  else:
    return vms, False


In [28]:
def powerConsumption(myVmSpace):
    c = 5
    a = 100
    b = 200
    threshold = 0.5
    
    Pwr_dy = 0
    Pwr_st = 0
    Pwr = 0
    
    for core in myVmSpace[0]:
        coreUsage = 1-core/5
        if(coreUsage > 0):
            Pwr_st = c
        if(coreUsage < threshold):
            Pwr_dy = a*coreUsage
        else:
            Pwr_dy = a*threshold + b*(coreUsage-threshold)*(coreUsage-threshold)
        Pwr = Pwr + Pwr_dy + Pwr_st
    return Pwr

In [6]:
def RR(taskQueue,myVmSpace):
               
    VM_iter = 0
    #will iterate through all possible starting VMs, incriments by one each time
    VM_jter = 0
    #incrimenter used when task does not fit in the first potentially available VM
    
    rejectQueue = []
               
    for i in taskQueue:
        if(df['NrmlTaskCores'][i] <= myVmSpace[0][VM_iter] and df['NrmlTaskMem'][i] <= myVmSpace[1][VM_iter]):
            #if the number of cores and memory space the task requires is less than the available for the VMS
            #then we can allocate the task to the current VM
            myVmSpace[0][VM_iter]  = myVmSpace[0][VM_iter] - df['NrmlTaskCores'][i]
            myVmSpace[1][VM_iter]  = myVmSpace[1][VM_iter] - df['NrmlTaskMem'][i]
            #now that the task is allocated, we have to update the amount of free CPU and memory space
        else:
            allocated = False
            VM_jter = VM_iter+1
            #move to the next VM
            if(VM_jter == 100):
                #make sure to loop back
                VM_jter = 0
            while(not allocated and VM_jter!=VM_iter):
                #if it is allocated then don't keep looking
                #if you've looked at all possible VMs and its not allocated, then it never will be so end loop
                if(df['NrmlTaskCores'][i] <= myVmSpace[0][VM_jter] and df['NrmlTaskMem'][i] <= myVmSpace[1][VM_jter]):
                    #if the number of cores and memory space the task requires is less than the available for the VMS
                    #then we can allocate the task to the current VM
                    myVmSpace[0][VM_jter] = myVmSpace[0][VM_jter] - df['NrmlTaskCores'][i]
                    myVmSpace[1][VM_jter] = myVmSpace[1][VM_jter] - df['NrmlTaskMem'][i]
                    #now that the task is allocated, we have to update the amount of free CPU and memory space
                    allocated = True
                    #allocate the task 
                VM_jter = VM_jter+1
                #move to the next VM if its not allocated to the last one
                if(VM_jter == 100):
                    VM_jter = 0
            if(not allocated):
                rejectQueue.append(df['TaskID'][i])
                #if the while loop finishes and the task is still not allocated
                #then it cycled through all VMs and did not fit any, so it cannot be allocated and should be added to reject queue
        VM_iter = VM_iter+1
        if(VM_iter == 100):
            VM_iter = 0

    return rejectQueue

# Greedy Algorithm 2 ways
* Optimize energy consumption
* Optimize cost

In [27]:
# def greedy(task_queue, vms, optIdentifier, currTime):
#     reject_queue = []
#     time_quantum = 300

#     all_past_threshold = False
#     vm_iter = 0
#     task_rejected = False

#     for task_idx in task_queue: 
#         minOptDelta = 100000000000
#         min_vm_idx = 0
#         vm_space_found = False
#         if task_rejected:
#           reject_queue.append(task['TaskID'])
#           continue

#         if not all_past_threshold and optIdentifier == "power":
#           vms, allocated = powerVMAlloc(vms,task_idx,vm_iter,all_past_threshold)
#           if not allocated and vm_iter == 99:
#               vm_iter = 0
#               all_past_threshold = True
#           elif not allocated and vm_iter < 99:
#               vm_iter += 1
#               vms, allocated = powerVMAlloc(vms,task_idx,vm_iter,all_past_threshold)
#               startTime = df['Time'][task_idx]
#               exTime = df['ExecutionTime'][task_idx]
#               if currTime == startTime + exTime - time_quantum:
#                 # Turnaround time is the curr time that the task finishes minus the arrival time
#                 finishTime = currTime + time_quantum
#                 df['TurnaroundTime_Power'][task_idx] = finishTime - startTime
#           else:
#             startTime = df['Time'][task_idx]
#             exTime = df['ExecutionTime'][task_idx]
#             if currTime == startTime + exTime - time_quantum:
#               # Turnaround time is the curr time that the task finishes minus the arrival time
#               finishTime = currTime + time_quantum
#               df['TurnaroundTime_Power'][task_idx] = finishTime - startTime
        
#         if all_past_threshold and optIdentifer == "power":
#           vms, allocated = powerVMAlloc(vms,task_idx,vm_iter,all_past_threshold)
#           if not allocated:
#             vm_jter = (vm_iter + 1) % 100
#             allocated = False
#             while vm_jter != vm_iter:
#               vms, allocated = powerVMAlloc(vms,task_idx,vm_jter,all_past_threshold)
#               if allocated:
#                 break
#               vm_jter = (vm_iter + 1) % 100
#             if not allocated:
#               rejectQueue.append(task['TaskID'])
#               task_rejected = True
#               continue
#           vm_iter += 1
#           startTime = df['Time'][task_idx]
#           exTime = df['ExecutionTime'][task_idx]
#           if currTime == startTime + exTime - time_quantum:
#             # Turnaround time is the curr time that the task finishes minus the arrival time
#             finishTime = currTime + time_quantum
#             df['TurnaroundTime_Power'][task_idx] = finishTime - startTime
        
#         if not all_past_threshold and optIdentifier == "cost":
#           # TODO: CHANGE FOR NEW COST CODE SETUP
#           powerDelta = powerVMAlloc(vms,task_idx,vm_idx)
#           prcIdx = int((currTime - 90000)/3600)
#           optDelta = Price_t[prcIdx]*powerDelta
#           if(optDelta < minOptDelta):
#               minOptDelta = optDelta
#               min_vm_idx = vm_idx
#           # Make sure you first run the code that initializes the execution time / turnaround
#           # dataframe before running the greedy algorithm! Do this every time
#           startTime = df['Time'][task_idx]
#           exTime = df['ExecutionTime'][task_idx]
#           if(currTime == startTime + exTime - time_quantum):
#               # Turnaround time is the curr time that the task finishes minus the arrival time
#               finishTime = currTime + time_quantum
#               df['TurnaroundTime_Cost'][task_idx] = finishTime - startTime
        
#         if all_past_threshold and optIdentifier == "cost":
#           # TODO: CHANGE FOR NEW COST CODE SETUP
#           powerDelta = powerVMAlloc(vms,task_idx,vm_idx)
#           prcIdx = int((currTime - 90000)/3600)
#           optDelta = Price_t[prcIdx]*powerDelta
#           if(optDelta < minOptDelta):
#               minOptDelta = optDelta
#               min_vm_idx = vm_idx
#           # Make sure you first run the code that initializes the execution time / turnaround
#           # dataframe before running the greedy algorithm! Do this every time
#           startTime = df['Time'][task_idx]
#           exTime = df['ExecutionTime'][task_idx]
#           if(currTime == startTime + exTime - time_quantum):
#               # Turnaround time is the curr time that the task finishes minus the arrival time
#               finishTime = currTime + time_quantum
#               df['TurnaroundTime_Cost'][task_idx] = finishTime - startTime

#     return reject_queue, vms


# Shortest Job First Algorithm
The Shortest Job First algorithm is optimal for waiting time and thus turnaround time (assuming task execution time is time-invariant). The challenge in most cases is not knowing the task execution time before execution. However, in this example, we know each task's execution time in advance. 
# There are several steps involved in implimenting SJF. 
1. Create a dictionary containing all vms for all times for which the algorithm will be run. Creating all vms at once allows us to allocate a process to VMs for all of its execution time, not just the current time quantum. 
2. Sort tasks arriving at the current time in a list in ascending order. This way, if we iterate through the sorted tasks, priority will be given to shorter tasks. 
3. Iterate through the sorted tasks implimenting a round robin-like algorithm. If a task can fit in the first VM, then allocate it. Otherwise, go through the next VMs until you find one it can fit in. If there are no VMs that the task can fit in, then append it to the reject queue. Always start the next task at the VM after the last one that was filled. 
4. return the reject queue and vm dictionary. 


In [51]:
def sjf(time_params,time_quantum,vm_size):
    vms = {} #vms for all possible times
    rejectQueue = []
    # Reinitialize Turnaround times
    df["TurnaroundTime"] = [0]*len(df)
    # Worst case:
    # If setup == 1, vms will be used at the max arrival time + time quantum * 2 (b/c burst time max = 600 and time quantum = 300)
    # If setup == 2, max time for vms will be max arrival time + time quantum * 10 (b/c burst time max = 3000 and time quantum = 300)
    counter = 0
    for time in range(time_params[0],time_params[1] + time_quantum * 2,time_quantum):
        vms[time] = [100*[vm_size[0]],100*[vm_size[1]]]
    for time in range(time_params[0],time_params[1],time_quantum):
        # print(time)
        df_t = df[df['Time'].isin([time])] #get the elements from a certain time
        print(df_t)
        # print("Length of df for time " + str(time) + " is " + str(len(df_t)))
        vm_iter = 0
        task_rejected = 0

        print("time is {}".format(time))
        print("number of tasks is {}".format(len(df_t)))

        for index,task in df_t.iterrows():
            if(task_rejected == 0):
                if(task["ExecutionTime"] == 300):
                    counter = counter + 1
                    # print("Entered inner for loop, index = " + str(index) + " counter = " + str(ctr) + " time = " + str(time))
                    if(task['NrmlTaskCores'] <= vms[time][0][vm_iter]) and (task['NrmlTaskMem'] <= vms[time][1][vm_iter]):
                        vms[time][0][vm_iter] = vms[time][0][vm_iter] - task['NrmlTaskCores']
                        vms[time][1][vm_iter] = vms[time][1][vm_iter] - task['NrmlTaskMem']
                        finTime = time + 300
                        df.loc[[index], ["TurnaroundTime"]] = finTime - task["Time"]
                    else: 
                        vm_iter = vm_iter+1
                        if(vm_iter < len(vms[time][0])):
                            vms[time][0][vm_iter] = vms[time][0][vm_iter] - task['NrmlTaskCores']
                            vms[time][1][vm_iter] = vms[time][1][vm_iter] - task['NrmlTaskMem']
                            finTime = time + 300
                            df.loc[[index], ["TurnaroundTime"]] = finTime - task["Time"]
                        else:
                            task_rejected = 1
                            rejectQueue.append(task['TaskID'])
            else:
                rejectQueue.append(task['TaskID'])
        for index,task in df_t.iterrows():
            if(task_rejected == 0):
                if(task["ExecutionTime"] == 600):
                    counter = counter + 1
                    # print("Entered inner for loop, index = " + str(index) + " counter = " + str(ctr) + " time = " + str(time))
                    if(task['NrmlTaskCores'] <= vms[time][0][vm_iter]) and (task['NrmlTaskMem'] <= vms[time][1][vm_iter]):
                        vms[time][0][vm_iter] = vms[time][0][vm_iter] - task['NrmlTaskCores']
                        vms[time][1][vm_iter] = vms[time][1][vm_iter] - task['NrmlTaskMem']
                        vms[time+300][0][vm_iter] = vms[time+300][0][vm_iter] - task['NrmlTaskCores']
                        vms[time+300][1][vm_iter] = vms[time+300][1][vm_iter] - task['NrmlTaskMem']
                        finTime = time + 600
                        idx = list(df["TaskID"]).index(task["TaskID"])
                        df["TurnaroundTime"][idx] = finTime - df["Time"][idx]
                    else: 
                        vm_iter = vm_iter+1
                        if(vm_iter < len(vms[time][0])):
                            vms[time][0][vm_iter] = vms[time][0][vm_iter] - task['NrmlTaskCores']
                            vms[time][1][vm_iter] = vms[time][1][vm_iter] - task['NrmlTaskMem']
                            vms[time+300][0][vm_iter] = vms[time+300][0][vm_iter] - task['NrmlTaskCores']
                            vms[time+300][1][vm_iter] = vms[time+300][1][vm_iter] - task['NrmlTaskMem']
                            finTime = time + 600
                            idx = list(df["TaskID"]).index(task["TaskID"])
                            df["TurnaroundTime"][idx] = finTime - df["Time"][idx]
                        else:
                            task_rejected = 1
            else:
                rejectQueue.append(task['TaskID'])
                
        print("time is {}".format(time))
        print("count is {}".format(counter))
    return vms,rejectQueue


In [54]:
def compare(leftIdx, rightIdx):
  if df['NrmlTaskCores'][leftIdx] < df['NrmlTaskCores'][rightIdx]:
    return -1
  elif df['NrmlTaskCores'][leftIdx] == df['NrmlTaskCores'][rightIdx]:
    return 0
  else:
    return 1


def generateTaskQueues():
    taskQueues = {}
    for time in range(90000,113100,300):
        taskQueues[time] = []
    time = 90000
    for taskIdx in range(len(df['Time'])):
        if(df['Time'][taskIdx] == time):
            exTime = df['ExecutionTime'][taskIdx]
            timeCnt = 0
            while(exTime > 0):
                taskQueues[time+timeCnt].append(taskIdx)
                timeCnt = timeCnt + 300
                exTime = exTime - 300
        else:
            # Sort task queue for current time before going to next time
            taskQueues[time] = sorted(taskQueues[time], key=cmp_to_key(compare), reverse=False)
            time = time + 300
        
    return taskQueues

In [ ]:
exec_time_arr = []
subtasks = 0
quantum = 300
rejectQueue = []
burstTime = None
for index,task in df.iterrows():
    taskID = task["TaskID"]
    burstTime = ((taskID%2)+1)*quantum      # only two options for burst time
    exec_time_arr.append(burstTime)
    subtasks += burstTime / quantum

time_dict = {
    'ExecutionTime': exec_time_arr,
    'TurnaroundTime_Power': [0]*len(df["Time"]),
    'TurnaroundTime_Cost': [0]*len(df["Time"]),
    'TurnaroundTime': [0]*len(df["Time"])
}
for key in time_dict.keys():
    df[key] = time_dict[key]
print("Subtasks:", subtasks)

# Create power and cost dictionaries
Pwr_t = {}
Cost_t = {}

Price_t = [0.5, 0.5, 0.6, 0.6, 0.6, 0.7, 0.7, 0.6, 0.6, 0.8, 0.8, 0.8, 0.8]

for time in range(90000,113100,300): 
    Pwr_t[time] = []
    Cost_t[time] = []

taskQueues = generateTaskQueues()

rejectQueue = []

In [39]:
def greedy(task_queue, vms, optIdentifier, currTime):
    reject_queue = []
    time_quantum = 300

    all_past_threshold = False
    vm_iter = 0
    task_rejected = False

    for task_idx in task_queue: 
        if task_rejected:
          reject_queue.append(task['TaskID'])
          continue

        # If not all vms have reached past the given threshold (0.5 or 50% in this case), allocate
        # as so: keep allocating tasks to one vm until that vm reaches the threshold.
        # When the vm reaches the threshold, allocate the task to the next vm and continue
        # allocating to the next vm. Repeat this process until all vms reach past the threshold
        if not all_past_threshold and optIdentifier == "power":
          vms, allocated = powerVMAlloc(vms,task_idx,vm_iter,all_past_threshold)
          if not allocated and vm_iter == 99:
              vm_iter = 0
              all_past_threshold = True
              # Set all_past_threshold to true and then try to allocate the task again with
              # a RR approach (i.e. the if all_past_threshold and optIdentifier == "power" below this
              # will be executed)
          elif not allocated and vm_iter < 99:
              vm_iter += 1
              vms, allocated = powerVMAlloc(vms,task_idx,vm_iter,all_past_threshold)
              startTime = df['Time'][task_idx]
              exTime = df['ExecutionTime'][task_idx]
              if currTime == startTime + exTime - time_quantum:
                # Turnaround time is the curr time that the task finishes minus the arrival time
                finishTime = currTime + time_quantum
                df['TurnaroundTime_Power'][task_idx] = finishTime - startTime
          else:
            startTime = df['Time'][task_idx]
            exTime = df['ExecutionTime'][task_idx]
            if currTime == startTime + exTime - time_quantum:
              # Turnaround time is the curr time that the task finishes minus the arrival time
              finishTime = currTime + time_quantum
              df['TurnaroundTime_Power'][task_idx] = finishTime - startTime
        
        # If all the vms reach past the threshold, allocate as so: allocate the tasks
        # in a RR fashion. Because the tasks have been sorted by cpu units in ascending order,
        # allocating tasks RR will still keep power optimized. If a task can't be allocated to any vm,
        # we know all subsequent tasks cannot be allocated either (bc tasks are sorted by cpu units),
        # so just add the task and all subsequent tasks to the reject queue
        if all_past_threshold and optIdentifier == "power":
          vms, allocated = powerVMAlloc(vms,task_idx,vm_iter,all_past_threshold)
          if not allocated:
            vm_jter = (vm_iter + 1) % 100
            allocated = False
            while vm_jter != vm_iter:
              vms, allocated = powerVMAlloc(vms,task_idx,vm_jter,all_past_threshold)
              if allocated:
                break
              vm_jter = (vm_iter + 1) % 100
            if not allocated:
              rejectQueue.append(task['TaskID'])
              task_rejected = True
              continue
          vm_iter += 1
          startTime = df['Time'][task_idx]
          exTime = df['ExecutionTime'][task_idx]
          if currTime == startTime + exTime - time_quantum:
            # Turnaround time is the curr time that the task finishes minus the arrival time
            finishTime = currTime + time_quantum
            df['TurnaroundTime_Power'][task_idx] = finishTime - startTime
        
        if not all_past_threshold and optIdentifier == "cost":
          # TODO: CHANGE FOR NEW COST CODE SETUP
          powerDelta = powerVMAlloc(vms,task_idx,vm_idx)
          prcIdx = int((currTime - 90000)/3600)
          optDelta = Price_t[prcIdx]*powerDelta
          if(optDelta < minOptDelta):
              minOptDelta = optDelta
              min_vm_idx = vm_idx
          # Make sure you first run the code that initializes the execution time / turnaround
          # dataframe before running the greedy algorithm! Do this every time
          startTime = df['Time'][task_idx]
          exTime = df['ExecutionTime'][task_idx]
          if(currTime == startTime + exTime - time_quantum):
              # Turnaround time is the curr time that the task finishes minus the arrival time
              finishTime = currTime + time_quantum
              df['TurnaroundTime_Cost'][task_idx] = finishTime - startTime
        
        if all_past_threshold and optIdentifier == "cost":
          # TODO: CHANGE FOR NEW COST CODE SETUP
          powerDelta = powerVMAlloc(vms,task_idx,vm_idx)
          prcIdx = int((currTime - 90000)/3600)
          optDelta = Price_t[prcIdx]*powerDelta
          if(optDelta < minOptDelta):
              minOptDelta = optDelta
              min_vm_idx = vm_idx
          # Make sure you first run the code that initializes the execution time / turnaround
          # dataframe before running the greedy algorithm! Do this every time
          startTime = df['Time'][task_idx]
          exTime = df['ExecutionTime'][task_idx]
          if(currTime == startTime + exTime - time_quantum):
              # Turnaround time is the curr time that the task finishes minus the arrival time
              finishTime = currTime + time_quantum
              df['TurnaroundTime_Cost'][task_idx] = finishTime - startTime

    return reject_queue, vms

#power optimization
rejectQueue = []
for time in range(90000,113100,300): #113100
    taskQueue_t = taskQueues[time]
    vms = [100*[5],100*[10]]

    #reinitialize VMs for new time
    rejectQueue_t, vms_t = greedy(taskQueue_t,vms,"power", time)
    print("Finished running for time {}".format(time))
    if(rejectQueue_t != []):
        rejectQueue.extend(rejectQueue_t)
    Pwr_t[time] = powerConsumption(vms_t)
    totalPwr = totalPwr + Pwr_t[time]
    prcIdx = int((time - 90000)/3600)
    Cost_t[time] = Price_t[prcIdx]*Pwr_t[time]
    totalCost = totalCost + Cost_t[time]

print("Power optimization:")
print("\tTotal power:", totalPwr)
print("\tTotal cost: ", totalCost)

turnaroundtime_sum = 0
for index,task in df.iterrows():
    turnaroundtime_sum += task.TurnaroundTime_Power
avg_turnaroundtime = float(turnaroundtime_sum) / float(len(df))
print("\tAverage turnaround time:", avg_turnaroundtime)

print("\t" + str(len(rejectQueue)) + " rejections")
if (len(rejectQueue) > 0):
    rejectFileName = "taskReject_1_power.npy"
    np.save(rejectFileName, rejectQueue)
vmFileName = "VMs_1_power.npy"
np.save(vmFileName, vms)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Finished running for time 90000


KeyboardInterrupt: ignored

In [ ]:
#cost optimization
totalPwr = 0
totalCost = 0
rejectQueue = []
for time in range(90000,113100,300):
    taskQueue_t = taskQueues[time]
    vms = [100*[5],100*[10]]
    #reinitialize VMs for new time
    rejectQueue_t, vms_t = greedy(taskQueue_t,vms,"cost",time)
    if(rejectQueue_t != []):
        rejectQueue.extend(rejectQueue_t)
    Pwr_t[time] = powerConsumption(vms_t)
    totalPwr = totalPwr + Pwr_t[time]
    prcIdx = int((time - 90000)/3600)
    Cost_t[time] = Price_t[prcIdx]*Pwr_t[time]
    totalCost = totalCost + Cost_t[time]

print("Cost optimization:")
print("\tTotal power:", totalPwr)
print("\tTotal cost: ", totalCost)

turnaroundtime_sum = 0
for index,task in df.iterrows():
    turnaroundtime_sum += task.TurnaroundTime_Cost
avg_turnaroundtime = float(turnaroundtime_sum) / float(len(df))
print("\tAverage turnaround time:", avg_turnaroundtime)

print("\t" + str(len(rejectQueue)) + " rejections")
if (len(rejectQueue) > 0):
    rejectFileName = "taskReject_1_cost.npy"
    np.save(rejectFileName, rejectQueue)
vmFileName = "VMs_1_cost.npy"
np.save(vmFileName, vms)

In [19]:
# Turnaround time optimization
totalPwr = 0
totalCost = 0
rejectQueue = []
vms,rejectQueue = sjf([90000,113100],300,[5,10])
for time in range(90000,113100,300):
    vms_t = vms[time]
    Pwr_t[time] = powerConsumption(vms_t)
    totalPwr = totalPwr + Pwr_t[time]
    prcIdx = int((time - 90000)/3600)
    Cost_t[time] = Price_t[prcIdx]*Pwr_t[time]
    totalCost = totalCost + Cost_t[time]

print("Turnaround Time optimization:")
print("\tTotal power:", totalPwr)
print("\tTotal cost: ", totalCost)

turnaroundtime_sum = 0
for index,task in df.iterrows():
    turnaroundtime_sum += task.TurnaroundTime
avg_turnaroundtime = float(turnaroundtime_sum) / float(len(df))
print("\tAverage turnaround time:", avg_turnaroundtime)

print("\t" + str(len(rejectQueue)) + " rejections")
if (len(rejectQueue) > 0):
    rejectFileName = "taskReject_1_turnaround.npy"
    np.save(rejectFileName, rejectQueue)
vmFileName = "VMs_1_turnaround.npy"
np.save(vmFileName, vms)

      Time    ParentID  ...  TurnaroundTime_Cost  TurnaroundTime
0    90000   757745334  ...                    0               0
1    90000   975992247  ...                    0               0
2    90000  1468458091  ...                    0               0
3    90000  1460281235  ...                    0               0
4    90000  1164728954  ...                    0               0
..     ...         ...  ...                  ...             ...
591  90000  1288997448  ...                    0               0
592  90000   757745334  ...                    0               0
593  90000  1488482422  ...                    0               0
594  90000  1263655469  ...                    0               0
595  90000   679429735  ...                    0               0

[596 rows x 11 columns]
time is 90000
number of tasks is 596
time is 90000
count is 596


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


        Time    ParentID  ...  TurnaroundTime_Cost  TurnaroundTime
596    90300  1471393173  ...                    0               0
597    90300  1321294100  ...                    0               0
598    90300  1444778069  ...                    0               0
599    90300  1447958256  ...                    0               0
600    90300  1106992769  ...                    0               0
...      ...         ...  ...                  ...             ...
49056  90300  1433362210  ...                    0               0
49057  90300  1488535799  ...                    0               0
49058  90300  1488535814  ...                    0               0
49059  90300  1488535799  ...                    0               0
49060  90300  1488482422  ...                    0               0

[48465 rows x 11 columns]
time is 90300
number of tasks is 48465


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:65: SettingWithCopyWarning: 
A value is trying to b

time is 90300
count is 47941
         Time    ParentID  ...  TurnaroundTime_Cost  TurnaroundTime
49061   90600  1471393173  ...                    0               0
49062   90600  1321294100  ...                    0               0
49063   90600  1444778069  ...                    0               0
49064   90600   783927992  ...                    0               0
49065   90600  1447958256  ...                    0               0
...       ...         ...  ...                  ...             ...
101736  90600  1478398502  ...                    0               0
101737  90600   981883546  ...                    0               0
101738  90600   975992247  ...                    0               0
101739  90600  1263655469  ...                    0               0
101740  90600  1263655469  ...                    0               0

[52680 rows x 11 columns]
time is 90600
number of tasks is 52680


...

time is 111900
count is 2905199
           Time    ParentID  ...  TurnaroundTime_Cost  TurnaroundTime
3482798  112200  1471393173  ...                    0               0
3482799  112200  1321294100  ...                    0               0
3482800  112200  1444778069  ...                    0               0
3482801  112200  1433362006  ...                    0               0
3482802  112200  1447958256  ...                    0               0
...         ...         ...  ...                  ...             ...
3530353  112200  1489018523  ...                    0               0
3530354  112200  1489018523  ...                    0               0
3530355  112200  1489018523  ...                    0               0
3530356  112200  1489018523  ...                    0               0
3530357  112200  1489018523  ...                    0               0

[47560 rows x 11 columns]
time is 112200
number of tasks is 47560


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:65: SettingWithCopyWarning: 
A value is trying to b

time is 112200
count is 2946348
           Time    ParentID  ...  TurnaroundTime_Cost  TurnaroundTime
3530358  112500  1471393173  ...                    0               0
3530359  112500  1321294100  ...                    0               0
3530360  112500   754112966  ...                    0               0
3530361  112500   679429735  ...                    0               0
3530362  112500  1447958256  ...                    0               0
...         ...         ...  ...                  ...             ...
3535024  112500  1487094655  ...                    0               0
3535025  112500  1461321601  ...                    0               0
3535026  112500  1487094655  ...                    0               0
3535027  112500   618817162  ...                    0               0
3535028  112500  1213243701  ...                    0               0

[4671 rows x 11 columns]
time is 112500
number of tasks is 4671


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


time is 112500
count is 2951019
Empty DataFrame
Columns: [Time, ParentID, TaskID, JobType, NrmlTaskCores, NrmlTaskMem, Unnamed: 6, ExecutionTime, TurnaroundTime_Power, TurnaroundTime_Cost, TurnaroundTime]
Index: []
time is 112800
number of tasks is 0
time is 112800
count is 2951019
Turnaround Time optimization:
	Total power: 579013.7452343736
	Total cost:  343176.9000234366
	Average turnaround time: 350.6137007645482
	1169352 rejections
